## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

Add our `utils` directory to the system's `$PATH` so we can import Python files from sibling directories.

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import requests
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata
import regex as re
from datetime import datetime
from dateutil.parser import parse

## Download

Retrieve the page

In [6]:
url = "https://www.amadorgov.org/services/covid-19/-fsiteid-1"

In [7]:
page = requests.get(url)

In [8]:
soup = BeautifulSoup(page.content, "html.parser")

Get content well

In [9]:
content = soup.find("div", id="widget_4_3778_3488")

Get timestamp

In [10]:
# imgs = [
#     c
#     for c in content.find_all("img")
#     if "Screenshot" and "Top" in c.attrs.get("alt", "")
# ]

In [11]:
imgs = content.find_all("img")

AttributeError: 'NoneType' object has no attribute 'find_all'

In [15]:
imgs

[<img alt="COVID-19 image 4.21.20" height="373" src="/home/showpublishedimage/4397/637230569666270000" style="width: 469px; height: 180px;" width="810"/>,
 <img alt="" src="/home/showpublishedimage/4893" style="width: 650px; height: 797px;"/>,
 <img alt="Stats 02.17 - Bottom" src="/home/showpublishedimage/4895" style="margin: 15px; width: 850px; height: 790px;"/>,
 <img alt="Road_to Recovery Resources label" src="/home/showpublishedimage/4431/637246532725070000" style="width: 380px; height: 254px;"/>,
 <img alt="Press Releases" src="/home/showpublishedimage/3409/637229943791030000" style="width: 330px; height: 185px;"/>,
 <img alt="3D Coronavirus_Emergency Declarations" src="/home/showpublishedimage/3413/637229944542970000" style="width: 330px; height: 185px;"/>,
 <img alt="3D Coronavirus image_Where Can I Get Tested" src="/home/showpublishedimage/4445/637251359468630000" style="width: 330px; height: 186px;"/>,
 <img alt="How can I Protect Myself" src="/home/showpublishedimage/3425/637

In [16]:
url = f"https://www.amadorgov.org{imgs[1]['src']}"

In [17]:
tz = pytz.timezone("America/Los_Angeles")

In [18]:
today = datetime.now(tz).date()

In [19]:
with open(data_dir / "amador" / f"{today}.png", "wb") as handler:
    handler.write(requests.get(url).content)